In [1]:
import sys
import os
sys.path.append(os.path.abspath("code"))
# these are  needed-
import pandas as pd
import numpy as np
from GnomAD_df_class import GnomAD_df

# main parquet update notebook
In this notebook I'll writte the updated on the main parquet file which contains the samples variants, intervals & allele frequency <br>

2/3/2023
The main file contains information of the following samples: <br>
#### Sinclair's samples:
1. AS22WG001
2. AS22WG002
3. AS22WG003
4. AS22WG004
5. AS22WG005
6. AS22WG006
7. AS22WG007
8. AS22WG008
10. AS22WG009 
11. AS22WG010
12. AS22WG011
13. AS22WG012
14. AS22WG013
15. AS22WG014
16. AS22WG015
17. AS22WG016
18. AS22WG017
19. AS22WG018
20. AS22WG019
21. AS22WG020
22. AS22WG021
23. AS22WG022 
24. AS22WG023
25. AS22WG024


#### Anu & Ken's samples: 
##### raw data samples: 
The following samples data was retrived from the raw vcf file, the processed files will be added on arrival
1. PKGen162 

2. PKGen160 
3. PKGen164 <br>


##### SNP data:

1. PKGen169 
2. PKGen176 
3. PKGen171 
4. PKGen185 
5. PKGen182 
6. PKGen166 
7. PKGen177 
8. PKGen186 
9. PKGen172 
10. PKGen187

On the 1/3/2023 we noticed some of the samples do not have gnomAD's alleles frequencey (AF), Although it can be found on the website. <br>

In order to fix it, the script 'get_gnomAD_AF.sh', was applied. it can be found under "code folder   <br>
to run it, the following command was applied
>```bash
>sbatch --mem 8G --killable --requeue -c 4 --array=0-23 -t 72:00:00 --wrap "code/get_gnomAD_AF.sh"
>```

<br>
The job is killable due the useage of all the lab resources on the cluster. it means the job have low priority. <br>
The following chromosmes ended the scripts without errors: <br>

- chr4
- chr6
- chr7
- chr20
- chr21
- chr22
- chrX
- chrY

while the reset reported **bgzf_read_block**. The "bgzf_read_block" error in bcftools usually indicates a problem with reading compressed data blocks in a BGZF file. <br>
there are two possible explanation for the error: <br> 
1. corrupt file
2. insufficient disk space or memory errors

Since we are using a file which is largely in use worldwide, the 2nd option is more likely, killable flag or insufficient memory given to the command.<br>

In the following part, I'll compare between the current AF to the new generated. <br>

In [10]:
no_error_chr = ["chr4", "chr6", "chr7", "chr20", "chr21", "chr22", "chrX", "chrY"]
error_chr = ["chr" + str(i) for i in range(1,20) if "chr" + str(i) not in  no_error_chr ]
def get_new_chrom_variant(chrom):
    """
    opens the new generated AF variant file 
    """
    df = pd.read_csv(f'data/{chrom}_gnomAD_AF.tsv', sep='\t', header=None).loc[:,:4]
    df.columns = ['CHROM','POS','REF','ALT','NEW_AF']
    return df.set_index(['CHROM','POS','REF','ALT'])
    
# loading the current DF
current_df = GnomAD_df('data/main.prq', only_peak_variants=True, remove_unkown=False, remove_phased_gt=False).get_table()
current_df.head()

In [28]:
# creating a table which contains the current AF column and the new AF column

AF_df = current_df[['AF']]
AF_df['NEW_AF'] = np.nan
for chrom in no_error_chr + error_chr:
    df = get_new_chrom_variant(chrom)
    df = df[df.index.isin(current_df.index)]
    AF_df.loc[df.index,'NEW_AF'] = df.NEW_AF.values
AF_df = AF_df.reset_index()

In [33]:
print(f"# of NaN in current table - {AF_df.AF.isna().sum()}\n# of NaN in new AF - {AF_df.NEW_AF.isna().sum()}")
nona = AF_df.dropna()
print(f"# of variants AF in current df - {AF_df.AF.notna().sum()}\n# of variants AF in new df - {AF_df.NEW_AF.notna().sum()}\n# variants exist in both tables  {nona.shape[0]}")

# of NaN in current table - 172
# of NaN in new AF - 132914
# of variants AF in current df - 134137
# of variants AF in ne df - 1395
# variants exist in both tables  1388


The big difference in the numbers might be cause due the error I got in the generating script <br>
Now we will look only on chromosome which the script fully finished